In [4]:
import os

os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [5]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql.window import Window
import time
import shutil
from pyspark.sql.functions import current_timestamp, col, unix_timestamp

from datetime import datetime, timedelta


import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


In [6]:
# Định nghĩa schema của dữ liệu từ Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("binancecoin", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricess") \
    .option("startingOffsets", "latest") \
    .load()

# Chuyển đổi dữ liệu từ Kafka
parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data"))

# Lấy cột timestamp và bitcoin giá trị
crypto_parsed_df = parsed_df.select(
    to_timestamp(col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS").alias("DATE"),
    col("data.prices.bitcoin").alias("BTC_CLOSE"),
    col("data.prices.ethereum").alias("ETH_CLOSE"),
    col("data.prices.tether").alias("USDT_CLOSE"),
    col("data.prices.binancecoin").alias("BNB_CLOSE"),
    col("data.prices.usd-coin").alias("USDC_CLOSE"),
    col("data.prices.ripple").alias("XRP_CLOSE"),
    col("data.prices.cardano").alias("ADA_CLOSE"),
    col("data.prices.matic-network").alias("MATIC_CLOSE"),
    col("data.prices.dogecoin").alias("DOGE_CLOSE"),
    col("data.prices.solana").alias("SOL_CLOSE")
)

# Giới hạn dữ liệu chỉ trong 24 giờ qua
crypto_parsed_df = crypto_parsed_df.filter(
    unix_timestamp(current_timestamp()) - unix_timestamp(col("DATE")) <= 86400  # 86400 giây = 24 giờ
)

# Định nghĩa output và checkpoint
output_path = "gs://tmp-24h-data/crypto_streaming_data"
checkpoint_path = "gs://tmp-24h-data/checkpoint"

# Ghi dữ liệu vào GCS
query = crypto_parsed_df.writeStream \
    .format("parquet") \
    .option("checkpointLocation", checkpoint_path) \
    .outputMode("append") \
    .option("path", output_path) \
    .start()

# Chạy job liên tục
query.awaitTermination()


24/12/11 23:27:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/11 23:27:43 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/12/11 23:27:43 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/12/11 23:27:43 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/12/11 23:27:43 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/12/11 23:27:43 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
24/12/11 23:32:19 WARN FileStreamSinkLog: Compacting took 3028 ms for compact batch 9
24/12/11 23:37:26 WARN FileStreamSinkLog: Compacting took 3498 ms for compact batch 19
24/12/11 23:42:34 WARN FileStreamSinkLog: Compacting took 3430 ms for compact batch 29
2

KeyboardInterrupt: 